<a href="https://colab.research.google.com/github/pedrofranke/ProyectoFinal-Henry/blob/main/Data%20Engineering/Unified_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [2]:
google_path = '/content/drive/MyDrive/Henry - Proyecto Final/Datos PF Henry/Google/'
yelp_path = '/content/drive/MyDrive/Henry - Proyecto Final/Datos PF Henry/Yelp/'

In [3]:
google_reviews = pd.read_parquet(google_path+'google_reviews.parquet')
google_reviews.head()

,user_id,user_name,time,rating,text,restaurant_name,state,city,gmap_id
0,109434968607034492648,Tony Pinto,2013-11-18 14:03:54.547,5,I came by yesterday to pick up some pizza on t...,Raffaello Kosher Pizza,New York,New York,0x89c258ffaeaba947:0x8355860772a595a9
1,108584493264435262292,Benji,2015-02-17 22:27:28.498,1,Food was burnt and not good. You would think s...,Raffaello Kosher Pizza,New York,New York,0x89c258ffaeaba947:0x8355860772a595a9
2,108987049306954366378,Michael Rahmani,2014-12-30 22:43:47.508,1,This place is horrible. I ordered two ziti piz...,Raffaello Kosher Pizza,New York,New York,0x89c258ffaeaba947:0x8355860772a595a9
3,107767094884093007779,eliau piha,2014-07-06 23:26:13.836,5,Love their pizza and their service!\nREALLY IT...,Raffaello Kosher Pizza,New York,New York,0x89c258ffaeaba947:0x8355860772a595a9
4,107373453968933712290,Matt Schaffnit,2016-10-25 08:30:54.400,5,Used to get lunch here nearly every day. You w...,Raffaello Kosher Pizza,New York,New York,0x89c258ffaeaba947:0x8355860772a595a9


In [ ]:
google_restaurant = pd.read_parquet(google_path+'google_restaurant_clean.parquet')
google_restaurant.head()

,restaurant_name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130,4.4,18,0,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855,4.5,18,0,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42
2,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Texas,Austin,78702,30.273985,-97.719563,4.5,8,0,0x8644b59b8fe872e5:0x5e638876caa84cc3
3,Oneyda's Bakery,"Oneyda's Bakery, 600 Goodlette-Frank Rd #101, ...",Florida,Naples,34102,26.154754,-81.790528,4.6,19,1,0x88dae191ee505917:0x6ba3e25388d3fad4
4,Top Cat Seafood Restaurant,"Top Cat Seafood Restaurant, 3117 Martin Luther...",Texas,Dallas,75215,32.773130,-96.764484,3.9,8,0,0x864e9891e381f3df:0x4cefe6219bc9199c


In [4]:
google_reviews.drop(columns=['state','city','restaurant_name','user_name'],inplace=True)

In [ ]:
google_users = pd.read_parquet(google_path+'google_users.parquet')
google_users.head()

,user_id,user_name,review_count,average_rating
0,104819208193648646391,Gregor J. Rothfuss,102,3.950980
1,106654503918907830147,The Corcoran Group,99,4.060606
2,114955250538652050870,Javier Kohen,60,3.966667
3,115082761597075271038,ej shortell,46,4.130435
4,109673791694826464177,Jackie Gordon Singing Chef,44,3.863636


In [ ]:
google_dummies = pd.read_parquet(google_path+'google_rest_cat_final.parquet')
google_dummies.head()

,gmap_id,Predicted_Category,Counts
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Asian,1
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No Detail,1
2,0x8644b59b8fe872e5:0x5e638876caa84cc3,No Detail,1
3,0x88dae191ee505917:0x6ba3e25388d3fad4,Breakfast,1
4,0x864e9891e381f3df:0x4cefe6219bc9199c,Family,1


In [ ]:
google_dummies.drop(columns='Counts',inplace=True)

google_restaurant = pd.merge(google_restaurant,google_dummies,on='gmap_id',how='left')
google_restaurant.head()

,restaurant_name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,Predicted_Category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130,4.4,18,0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Asian
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130,4.4,18,0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Asian
2,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855,4.5,18,0,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No Detail
3,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855,4.5,18,0,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No Detail
4,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Texas,Austin,78702,30.273985,-97.719563,4.5,8,0,0x8644b59b8fe872e5:0x5e638876caa84cc3,No Detail


In [ ]:
google_restaurant.drop(columns='price_numeric',inplace=True)


In [ ]:
google_restaurant.columns

Index(['restaurant_name', 'address', 'state', 'city', 'postal_code',
       'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'gmap_id',
       'Predicted_Category'],
      dtype='object')

In [ ]:
restaurant_cols = ['business_name', 'address', 'state', 'city', 'postal_code',
       'latitude', 'longitude', 'avg_rating', 'review_count', 'business_id',
       'category']
google_restaurant.columns = restaurant_cols

google_restaurant.head()

,business_name,address,state,city,postal_code,latitude,longitude,avg_rating,review_count,business_id,category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130,4.4,18,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Asian
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130,4.4,18,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Asian
2,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855,4.5,18,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No Detail
3,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855,4.5,18,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No Detail
4,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Texas,Austin,78702,30.273985,-97.719563,4.5,8,0x8644b59b8fe872e5:0x5e638876caa84cc3,No Detail


In [ ]:
restaurant_ordered_cols = ['business_id','business_name','category', 'avg_rating',
                           'review_count', 'address', 'state', 'city', 'postal_code',
                           'latitude', 'longitude']

google_restaurant = google_restaurant[restaurant_ordered_cols]
google_restaurant.head()

,business_id,business_name,category,avg_rating,review_count,address,state,city,postal_code,latitude,longitude
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Asian,4.4,18,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130
1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Asian,4.4,18,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130
2,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,No Detail,4.5,18,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855
3,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,No Detail,4.5,18,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855
4,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,No Detail,4.5,8,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Texas,Austin,78702,30.273985,-97.719563


In [ ]:
google_restaurant.category.isna().sum()

0

In [ ]:
yelp_restaurant = pd.read_parquet(yelp_path+'df_business.parquet')
yelp_restaurant.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants"
9,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,6625 E 82nd St,Indianapolis,PA,46250,39.904320,-86.053080,4.0,20,"Steakhouses, Asian Fusion, Restaurants"
11,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,3001 Highway 31 W,White House,CA,37188,36.464747,-86.659187,3.5,8,"Pizza, Chicken Wings, Sandwiches, Restaurants"
12,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,3.0,41,"Pizza, Restaurants, Salad, Soup"


In [ ]:
yelp_restaurant.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'categories'],
      dtype='object')

In [ ]:
restaurant_cols = ['business_id', 'business_name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'avg_rating', 'review_count', 'category']

yelp_restaurant.columns = restaurant_cols
yelp_restaurant = yelp_restaurant[restaurant_ordered_cols]
yelp_restaurant.head()

,business_id,business_name,category,avg_rating,review_count,address,state,city,postal_code,latitude,longitude
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",4.0,80,935 Race St,CA,Philadelphia,19107,39.955505,-75.155564
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,"Food, Delis, Italian, Bakeries, Restaurants",4.5,100,2575 E Bay Dr,FL,Largo,33771,27.916116,-82.760461
9,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,"Steakhouses, Asian Fusion, Restaurants",4.0,20,6625 E 82nd St,PA,Indianapolis,46250,39.904320,-86.053080
11,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,"Pizza, Chicken Wings, Sandwiches, Restaurants",3.5,8,3001 Highway 31 W,CA,White House,37188,36.464747,-86.659187
12,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,"Pizza, Restaurants, Salad, Soup",3.0,41,4105 Main St,PA,Philadelphia,19127,40.022466,-75.218314


In [ ]:
yelp_dummies = pd.read_parquet(yelp_path+'yelp_rest_cat_final.parquet')
yelp_dummies.head()

,business_id,Predicted_Category
0,-0EdehHjIQc0DtYU8QcAig,Asian
1,-0FX23yAacC4bbLaGPvyxw,American
2,-0dKgi_Hpcis921nOpM85Q,American
3,-0eUa8TsXFFy0FCxHYmrjg,Cafe
4,-0gRYq5UjMtZbELj0KHxzA,Cafe


In [ ]:
yelp_restaurant = pd.merge(yelp_restaurant.drop(columns='category'),yelp_dummies,on='business_id',how='left')
yelp_restaurant.head()

,business_id,business_name,avg_rating,review_count,address,state,city,postal_code,latitude,longitude,Predicted_Category
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,4.0,80,935 Race St,CA,Philadelphia,19107,39.955505,-75.155564,Cafe
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,4.5,100,2575 E Bay Dr,FL,Largo,33771,27.916116,-82.760461,Family
2,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,4.0,20,6625 E 82nd St,PA,Indianapolis,46250,39.904320,-86.053080,Cafe
3,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,3.5,8,3001 Highway 31 W,CA,White House,37188,36.464747,-86.659187,American
4,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,3.0,41,4105 Main St,PA,Philadelphia,19127,40.022466,-75.218314,European


In [ ]:
yelp_restaurant.rename(columns={'Predicted_Category':'category'},inplace=True)
yelp_restaurant = yelp_restaurant[restaurant_ordered_cols]
yelp_restaurant.head()

,business_id,business_name,category,avg_rating,review_count,address,state,city,postal_code,latitude,longitude
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Cafe,4.0,80,935 Race St,CA,Philadelphia,19107,39.955505,-75.155564
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,Family,4.5,100,2575 E Bay Dr,FL,Largo,33771,27.916116,-82.760461
2,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,Cafe,4.0,20,6625 E 82nd St,PA,Indianapolis,46250,39.904320,-86.053080
3,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,American,3.5,8,3001 Highway 31 W,CA,White House,37188,36.464747,-86.659187
4,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,European,3.0,41,4105 Main St,PA,Philadelphia,19127,40.022466,-75.218314


In [ ]:
yelp_restaurant.category.unique()

array(['Cafe', 'Family', 'American', 'European', 'Asian',
       'Central American', 'Vegetarian', 'Fastfood', 'No Detail',
       'South American', 'Night', 'African', nan, 'Breakfast'],
      dtype=object)

In [ ]:
yelp_restaurant.category.fillna('No Detail',inplace=True)
yelp_restaurant.category.unique()

array(['Cafe', 'Family', 'American', 'European', 'Asian',
       'Central American', 'Vegetarian', 'Fastfood', 'No Detail',
       'South American', 'Night', 'African', 'Breakfast'], dtype=object)

In [ ]:
df_restaurant = pd.concat([yelp_restaurant,google_restaurant],ignore_index=True)
df_restaurant.head()

,business_id,business_name,category,avg_rating,review_count,address,state,city,postal_code,latitude,longitude
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Cafe,4.0,80,935 Race St,CA,Philadelphia,19107,39.955505,-75.155564
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,Family,4.5,100,2575 E Bay Dr,FL,Largo,33771,27.916116,-82.760461
2,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,Cafe,4.0,20,6625 E 82nd St,PA,Indianapolis,46250,39.904320,-86.053080
3,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,American,3.5,8,3001 Highway 31 W,CA,White House,37188,36.464747,-86.659187
4,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,European,3.0,41,4105 Main St,PA,Philadelphia,19127,40.022466,-75.218314


In [ ]:
df_restaurant.tail()

,business_id,business_name,category,avg_rating,review_count,address,state,city,postal_code,latitude,longitude
41557,0x89c2592cf2935ed9:0x6672264426649f94,Luthun,Family,4.8,68,"Luthun, 432 E 13th St, New York, NY 10009",New York,New York,10009,40.729911,-73.981883
41558,0x89c25a286e6ea2db:0x1291d4bbf41d5ed4,To Two Boonsik,Asian,4.3,45,"To Two Boonsik, 97 Canal St, New York, NY 10002",New York,New York,10002,40.715612,-73.993828
41559,0x89c25a27a5c98845:0xad087746b0c3381a,Matsunichi,Asian,4.2,48,"Matsunichi, 14-18 Elizabeth St #32-33, New Yor...",New York,New York,10013,40.715866,-73.997244
41560,0x89d31f176b64da79:0x202faca0f650e880,China King Express,Asian,4.1,118,"China King Express, 6938 Erie Rd, Derby, NY 14047",New York,Derby,14047,42.698185,-78.988253
41561,0x89d373386b718fff:0xf82c90ef48e7b23,House of Gourmet -- 食全食美,Asian,4.3,58,"House of Gourmet -- 食全食美, 2865 Sheridan Dr Sui...",New York,Tonawanda,14150,42.980251,-78.827337


In [ ]:
import Modulo as md

md.tipo_datos(df_restaurant)

,Columna,Tipo_datos,%_nulos,Nulos,Largo
0,business_id,[str],0.0,0,"[22, 37, 36, 35, 34, 33]"
1,business_name,[str],0.0,0,"[18, 20, 15, 14, 16, 24, 11, 31, 23, 26, 13, 1..."
2,category,[str],0.0,0,"[4, 6, 8, 5, 16, 10, 9, 14, 7]"
3,avg_rating,[float],0.0,0,[3]
4,review_count,[int],0.0,0,"[2, 3, 1, 4]"
5,address,[str],0.0,0,"[11, 13, 14, 17, 12, 20, 22, 30, 16, 31, 26, 1..."
6,state,[str],0.0,0,"[2, 10, 5, 7, 8, 12]"
7,city,[str],0.0,0,"[12, 5, 11, 6, 9, 7, 10, 15, 8, 16, 13, 19, 4,..."
8,postal_code,"[str, int]",0.0,0,"[5, 7, 3, 1, 4]"
9,latitude,[float],0.0,0,"[10, 13, 9, 12, 8, 7, 6, 11, 5, 18]"


In [ ]:
md.tipo_datos(yelp_restaurant)

,Columna,Tipo_datos,%_nulos,Nulos,Largo
0,business_id,[str],0.0,0,[22]
1,business_name,[str],0.0,0,"[18, 20, 15, 14, 16, 24, 11, 31, 23, 26, 13, 1..."
2,category,[str],0.0,0,"[4, 6, 8, 5, 16, 10, 9, 14, 7]"
3,avg_rating,[float],0.0,0,[3]
4,review_count,[int],0.0,0,"[2, 3, 1, 4]"
5,address,[str],0.0,0,"[11, 13, 14, 17, 12, 20, 22, 30, 16, 31, 26, 1..."
6,state,[str],0.0,0,[2]
7,city,[str],0.0,0,"[12, 5, 11, 6, 9, 7, 10, 15, 8, 16, 13, 19, 4,..."
8,postal_code,[str],0.0,0,"[5, 7, 3]"
9,latitude,[float],0.0,0,"[10, 13, 9, 12, 8, 7, 6, 11, 5]"


In [ ]:
yelp_restaurant.postal_code.unique()

array(['19107', '33771', '46250', ..., 'T8N 3X6', '08056', 'T5M 3L7'],
      dtype=object)

In [ ]:
md.tipo_datos(google_restaurant)

,Columna,Tipo_datos,%_nulos,Nulos,Largo
0,business_id,[str],0.0,0,"[37, 36, 35, 34, 33]"
1,business_name,[str],0.0,0,"[12, 13, 15, 26, 23, 11, 22, 81, 19, 28, 30, 6..."
2,category,[str],0.0,0,"[5, 9, 6, 10, 16, 8, 4, 14, 7]"
3,avg_rating,[float],0.0,0,[3]
4,review_count,[int],0.0,0,"[2, 1, 3, 4]"
5,address,[str],0.0,0,"[54, 55, 47, 62, 77, 60, 56, 119, 72, 61, 63, ..."
6,state,[str],0.0,0,"[10, 5, 7, 8, 12]"
7,city,[str],0.0,0,"[11, 9, 6, 8, 12, 10, 14, 5, 4, 13, 7, 15, 18,..."
8,postal_code,[int],0.0,0,"[5, 1, 4]"
9,latitude,[float],0.0,0,"[10, 9, 18, 8, 7, 6, 5]"


In [ ]:
df_restaurant.postal_code = df_restaurant.postal_code.apply(str)
md.tipo_datos(df_restaurant)

,Columna,Tipo_datos,%_nulos,Nulos,Largo
0,business_id,[str],0.0,0,"[22, 37, 36, 35, 34, 33]"
1,business_name,[str],0.0,0,"[18, 20, 15, 14, 16, 24, 11, 31, 23, 26, 13, 1..."
2,category,[str],0.0,0,"[4, 6, 8, 5, 16, 10, 9, 14, 7]"
3,avg_rating,[float],0.0,0,[3]
4,review_count,[int],0.0,0,"[2, 3, 1, 4]"
5,address,[str],0.0,0,"[11, 13, 14, 17, 12, 20, 22, 30, 16, 31, 26, 1..."
6,state,[str],0.0,0,"[2, 10, 5, 7, 8, 12]"
7,city,[str],0.0,0,"[12, 5, 11, 6, 9, 7, 10, 15, 8, 16, 13, 19, 4,..."
8,postal_code,[str],0.0,0,"[5, 7, 3, 1, 4]"
9,latitude,[float],0.0,0,"[10, 13, 9, 12, 8, 7, 6, 11, 5, 18]"


In [ ]:
save_path = '/content/drive/MyDrive/Henry - Proyecto Final/Datos PF Henry/Unified/'

df_restaurant.to_parquet(save_path+'df_restaurant.parquet')

In [ ]:
google_users.head()

,user_id,user_name,review_count,average_rating
0,104819208193648646391,Gregor J. Rothfuss,102,3.950980
1,106654503918907830147,The Corcoran Group,99,4.060606
2,114955250538652050870,Javier Kohen,60,3.966667
3,115082761597075271038,ej shortell,46,4.130435
4,109673791694826464177,Jackie Gordon Singing Chef,44,3.863636


In [ ]:
yelp_users = pd.read_parquet(yelp_path+'df_user.parquet')
yelp_users.head()

,user_id,name,review_count,useful,id_user_yelp
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,7217,0
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,43091,1
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2086,2
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,512,3
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,29,4


In [ ]:
yelp_users.drop(columns=['id_user_yelp','useful'],inplace=True)
yelp_users.columns = ['user_id','user_name','review_count']
yelp_users.head()

,user_id,user_name,review_count
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665
3,SZDeASXq7o05mMNLshsdIA,Gwen,224
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79


In [ ]:
google_users.drop(columns=['average_rating'],inplace=True)
google_users.head()

,user_id,user_name,review_count
0,104819208193648646391,Gregor J. Rothfuss,102
1,106654503918907830147,The Corcoran Group,99
2,114955250538652050870,Javier Kohen,60
3,115082761597075271038,ej shortell,46
4,109673791694826464177,Jackie Gordon Singing Chef,44


In [ ]:
md.tipo_datos(yelp_users)

,Columna,Tipo_datos,%_nulos,Nulos,Largo
0,user_id,[str],0.0,0,[22]
1,user_name,[str],0.0,0,"[6, 5, 4, 3, 8, 7, 9, 2, 11, 1, 10, 21, 13, 12..."
2,review_count,[int],0.0,0,"[3, 4, 2, 1, 5]"


In [ ]:
md.tipo_datos(google_users)

,Columna,Tipo_datos,%_nulos,Nulos,Largo
0,user_id,[str],0.0,0,[21]
1,user_name,[str],0.0,0,"[18, 12, 11, 26, 13, 10, 5, 15, 9, 14, 16, 8, ..."
2,review_count,[int],0.0,0,"[3, 2, 1]"


In [ ]:
df_users = pd.concat([yelp_users,google_users],ignore_index=True)
df_users.head()

,user_id,user_name,review_count
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665
3,SZDeASXq7o05mMNLshsdIA,Gwen,224
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79


In [ ]:
df_users.drop(columns=['user_name','review_count'],inplace=True)

We will not store amount of reviews, nor user_name because it is not meaningful and could represent liabilities.

In [9]:
google_reviews.head()

,user_id,time,rating,text,gmap_id
0,109434968607034492648,2013-11-18 14:03:54.547,5,I came by yesterday to pick up some pizza on t...,0x89c258ffaeaba947:0x8355860772a595a9
1,108584493264435262292,2015-02-17 22:27:28.498,1,Food was burnt and not good. You would think s...,0x89c258ffaeaba947:0x8355860772a595a9
2,108987049306954366378,2014-12-30 22:43:47.508,1,This place is horrible. I ordered two ziti piz...,0x89c258ffaeaba947:0x8355860772a595a9
3,107767094884093007779,2014-07-06 23:26:13.836,5,Love their pizza and their service!\nREALLY IT...,0x89c258ffaeaba947:0x8355860772a595a9
4,107373453968933712290,2016-10-25 08:30:54.400,5,Used to get lunch here nearly every day. You w...,0x89c258ffaeaba947:0x8355860772a595a9


In [10]:
google_reviews.columns

Index(['user_id', 'time', 'rating', 'text', 'gmap_id'], dtype='object')

In [11]:
review_list = ['user_id', 'date', 'rating', 'review', 'business_id']

google_reviews.columns = review_list


In [12]:
google_reviews['review_id'] = google_reviews['user_id'] + google_reviews['business_id']
google_reviews.columns

Index(['user_id', 'date', 'rating', 'review', 'business_id', 'review_id'], dtype='object')

In [13]:
ordered_review_columns = ['review_id', 'user_id', 'business_id', 'date', 'rating', 'review']
google_reviews = google_reviews[ordered_review_columns]
google_reviews.head()

,review_id,user_id,business_id,date,rating,review
0,1094349686070344926480x89c258ffaeaba947:0x8355...,109434968607034492648,0x89c258ffaeaba947:0x8355860772a595a9,2013-11-18 14:03:54.547,5,I came by yesterday to pick up some pizza on t...
1,1085844932644352622920x89c258ffaeaba947:0x8355...,108584493264435262292,0x89c258ffaeaba947:0x8355860772a595a9,2015-02-17 22:27:28.498,1,Food was burnt and not good. You would think s...
2,1089870493069543663780x89c258ffaeaba947:0x8355...,108987049306954366378,0x89c258ffaeaba947:0x8355860772a595a9,2014-12-30 22:43:47.508,1,This place is horrible. I ordered two ziti piz...
3,1077670948840930077790x89c258ffaeaba947:0x8355...,107767094884093007779,0x89c258ffaeaba947:0x8355860772a595a9,2014-07-06 23:26:13.836,5,Love their pizza and their service!\nREALLY IT...
4,1073734539689337122900x89c258ffaeaba947:0x8355...,107373453968933712290,0x89c258ffaeaba947:0x8355860772a595a9,2016-10-25 08:30:54.400,5,Used to get lunch here nearly every day. You w...


In [5]:
yelp_reviews = pd.read_parquet(yelp_path+'df_reviews.parquet')
yelp_reviews.head()

,business_id,review_id,user_id,stars,text,date,id_review_yelp
0,MTSW4McQd7CbVtyjqoe9mw,BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,4.0,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53,0
1,MTSW4McQd7CbVtyjqoe9mw,uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,4.0,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,1
2,MTSW4McQd7CbVtyjqoe9mw,a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,5.0,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57,2
3,MTSW4McQd7CbVtyjqoe9mw,MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5.0,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57,3
4,MTSW4McQd7CbVtyjqoe9mw,D1GisLDPe84Rrk_R4X2brQ,EouCKoDfzaVG0klEgdDvCQ,4.0,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35,4


In [6]:
yelp_reviews.drop(columns='id_review_yelp',inplace=True)
yelp_reviews.columns

Index(['business_id', 'review_id', 'user_id', 'stars', 'text', 'date'], dtype='object')

In [7]:
yelp_review_cols = ['business_id', 'review_id', 'user_id', 'rating', 'review', 'date']
yelp_reviews.columns = yelp_review_cols


In [14]:
yelp_reviews = yelp_reviews[ordered_review_columns]
yelp_reviews.head()

,review_id,user_id,business_id,date,rating,review
0,BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,MTSW4McQd7CbVtyjqoe9mw,2014-05-26 01:09:53,4.0,This is nice little Chinese bakery in the hear...
1,uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,MTSW4McQd7CbVtyjqoe9mw,2013-10-05 15:19:06,4.0,This is the bakery I usually go to in Chinatow...
2,a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,MTSW4McQd7CbVtyjqoe9mw,2013-10-25 01:34:57,5.0,"A delightful find in Chinatown! Very clean, an..."
3,MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,MTSW4McQd7CbVtyjqoe9mw,2018-05-20 17:58:57,5.0,I ordered a graduation cake for my niece and i...
4,D1GisLDPe84Rrk_R4X2brQ,EouCKoDfzaVG0klEgdDvCQ,MTSW4McQd7CbVtyjqoe9mw,2013-10-25 02:31:35,4.0,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...


In [ ]:
md.tipo_datos(yelp_reviews)

In [ ]:
md.tipo_datos(google_reviews)

In [16]:
google_reviews.rating = google_reviews.rating.apply(float)
md.tipo_datos(google_reviews)

In [17]:
df_reviews = pd.concat([yelp_reviews,google_reviews],ignore_index=True)
df_reviews.head()

,review_id,user_id,business_id,date,rating,review
0,BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,MTSW4McQd7CbVtyjqoe9mw,2014-05-26 01:09:53,4.0,This is nice little Chinese bakery in the hear...
1,uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,MTSW4McQd7CbVtyjqoe9mw,2013-10-05 15:19:06,4.0,This is the bakery I usually go to in Chinatow...
2,a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,MTSW4McQd7CbVtyjqoe9mw,2013-10-25 01:34:57,5.0,"A delightful find in Chinatown! Very clean, an..."
3,MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,MTSW4McQd7CbVtyjqoe9mw,2018-05-20 17:58:57,5.0,I ordered a graduation cake for my niece and i...
4,D1GisLDPe84Rrk_R4X2brQ,EouCKoDfzaVG0klEgdDvCQ,MTSW4McQd7CbVtyjqoe9mw,2013-10-25 02:31:35,4.0,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...


In [20]:
df_reviews[df_reviews.duplicated(subset=['user_id','business_id','date','review'],keep=False)].sort_values('review_id')

,review_id,user_id,business_id,date,rating,review
4341160,1000002928774164661590x88d9b6b280d37c49:0x51db...,100000292877416466159,0x88d9b6b280d37c49:0x51db4e7ff6550840,2017-09-19 19:59:20.025,2.0,The place is very nice. But. Very bad food and...
4341154,1000002928774164661590x88d9b6b280d37c49:0x51db...,100000292877416466159,0x88d9b6b280d37c49:0x51db4e7ff6550840,2017-09-19 19:59:20.025,2.0,The place is very nice. But. Very bad food and...
4031995,1000004331239598383010x88e5ba3aa20184c5:0xf5d6...,100000433123959838301,0x88e5ba3aa20184c5:0xf5d6f1dd76f48da4,2019-02-06 14:14:20.576,5.0,"Get the damn ribs, you will not regret it."
4032005,1000004331239598383010x88e5ba3aa20184c5:0xf5d6...,100000433123959838301,0x88e5ba3aa20184c5:0xf5d6f1dd76f48da4,2019-02-06 14:14:20.576,5.0,"Get the damn ribs, you will not regret it."
4065304,1000013623182976720530x88dca83babef63d1:0x526a...,100001362318297672053,0x88dca83babef63d1:0x526a61322a59dce,2016-05-14 13:34:36.089,5.0,Great place for breakfast. Finally real homefr...
...,...,...,...,...,...,...
1109984,QacdXke_oZAD662oeFyUxg,3B5wkOgsrjBjXRpGdrmTSA,n3Q7DXPyBzGjPK1l09SomQ,2021-02-01 17:04:06.000,4.0,"Fox Rehab at-home in Virgina-Northern, Fairfax..."
452331,gZU6syBqLk210QhnwWbSyw,6d4TXsdOZAsw8_h1r730uA,t1H2xPVsbTRu48bZ5TgItw,2010-12-27 23:22:50.000,1.0,I was home for the holidays and staying with m...
1109978,lzBHkMZayoGSpIPDOIIEug,3B5wkOgsrjBjXRpGdrmTSA,n3Q7DXPyBzGjPK1l09SomQ,2021-02-01 17:04:06.000,4.0,"Fox Rehab at-home in Virgina-Northern, Fairfax..."
1228370,qkbLeGVNwxY9-5iMDcwZGw,T870ZkavUI1rqaO_FrJFiw,mfTyBrvx9uCTSPjmrl8PjA,2008-08-27 09:49:33.000,5.0,Great hotel in the middle of the French quarte...


In [22]:
df_reviews.drop_duplicates(subset=['user_id','business_id','date','review','rating'],inplace=True)

In [23]:
df_reviews['year'] = df_reviews['date'].dt.year
df_reviews['month'] = df_reviews['date'].dt.month
df_reviews = df_reviews[df_reviews['year'] > 2015]
df_reviews = df_reviews[df_reviews['year'] < 2022]
df_reviews.year.unique()

array([2018, 2017, 2016, 2020, 2019, 2021])

In [25]:
df_reviews.reset_index(drop=True,inplace=True)
df_reviews.head()

,review_id,user_id,business_id,date,rating,review,year,month
0,3,3-1va0IQfK-9tUMzfHWfTA,MTSW4McQd7CbVtyjqoe9mw,2018-05-20 17:58:57,5.0,I ordered a graduation cake for my niece and i...,2018,5
1,5,KQSRUu4Aapl0hG6eu2v8iw,MTSW4McQd7CbVtyjqoe9mw,2018-03-17 23:47:25,4.0,This is my favorite bakery in Chinatown! I usu...,2018,3
2,7,qUfRCH5NUyRDsJfM6jA5PQ,MTSW4McQd7CbVtyjqoe9mw,2017-04-20 12:37:09,4.0,Impression: The good thing is that there are l...,2017,4
3,16,0q2W3-ieBUJWD5TTLKi3Ug,MTSW4McQd7CbVtyjqoe9mw,2016-04-12 17:35:26,4.0,"I would give five stars, but I probably didn't...",2016,4
4,17,z-yvbUGwFn8PAijEHdU_RA,MTSW4McQd7CbVtyjqoe9mw,2018-03-25 12:35:42,5.0,While I've never tried their bao (and usually ...,2018,3


In [26]:
df_reviews.review_id = df_reviews.index.tolist()
df_reviews.head()

,review_id,user_id,business_id,date,rating,review,year,month
0,0,3-1va0IQfK-9tUMzfHWfTA,MTSW4McQd7CbVtyjqoe9mw,2018-05-20 17:58:57,5.0,I ordered a graduation cake for my niece and i...,2018,5
1,1,KQSRUu4Aapl0hG6eu2v8iw,MTSW4McQd7CbVtyjqoe9mw,2018-03-17 23:47:25,4.0,This is my favorite bakery in Chinatown! I usu...,2018,3
2,2,qUfRCH5NUyRDsJfM6jA5PQ,MTSW4McQd7CbVtyjqoe9mw,2017-04-20 12:37:09,4.0,Impression: The good thing is that there are l...,2017,4
3,3,0q2W3-ieBUJWD5TTLKi3Ug,MTSW4McQd7CbVtyjqoe9mw,2016-04-12 17:35:26,4.0,"I would give five stars, but I probably didn't...",2016,4
4,4,z-yvbUGwFn8PAijEHdU_RA,MTSW4McQd7CbVtyjqoe9mw,2018-03-25 12:35:42,5.0,While I've never tried their bao (and usually ...,2018,3


In [27]:
df_reviews.drop(columns='date',inplace=True)
df_reviews.head()

,review_id,user_id,business_id,rating,review,year,month
0,0,3-1va0IQfK-9tUMzfHWfTA,MTSW4McQd7CbVtyjqoe9mw,5.0,I ordered a graduation cake for my niece and i...,2018,5
1,1,KQSRUu4Aapl0hG6eu2v8iw,MTSW4McQd7CbVtyjqoe9mw,4.0,This is my favorite bakery in Chinatown! I usu...,2018,3
2,2,qUfRCH5NUyRDsJfM6jA5PQ,MTSW4McQd7CbVtyjqoe9mw,4.0,Impression: The good thing is that there are l...,2017,4
3,3,0q2W3-ieBUJWD5TTLKi3Ug,MTSW4McQd7CbVtyjqoe9mw,4.0,"I would give five stars, but I probably didn't...",2016,4
4,4,z-yvbUGwFn8PAijEHdU_RA,MTSW4McQd7CbVtyjqoe9mw,5.0,While I've never tried their bao (and usually ...,2018,3


In [28]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

# Función para aplicar el análisis de sentimiento a un texto
def analyze_sentiment(text):
    if isinstance(text, str):
        compound_score = sia.polarity_scores(text)['compound']
        if compound_score >= 0.5:
            return 5
        elif compound_score >= 0:
            return 4
        elif compound_score >= -0.24:
            return 3
        elif compound_score >= -0.4:
            return 2
        else:
            return 1
    else:
        return 3  # Valor predeterminado si no es una cadena

# Vectorizar la función de análisis de sentimientos
vectorized_analyze_sentiment = np.vectorize(analyze_sentiment)

# Aplicar el análisis de sentimientos a la columna 'text'
df_reviews['sentiment_analysis'] = vectorized_analyze_sentiment(df_reviews['review'])

# Eliminar la columna 'text'
df_reviews.drop(columns=['review'],inplace=True)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [29]:
df_reviews.head()

,review_id,user_id,business_id,rating,year,month,sentiment_analysis
0,0,3-1va0IQfK-9tUMzfHWfTA,MTSW4McQd7CbVtyjqoe9mw,5.0,2018,5,5
1,1,KQSRUu4Aapl0hG6eu2v8iw,MTSW4McQd7CbVtyjqoe9mw,4.0,2018,3,5
2,2,qUfRCH5NUyRDsJfM6jA5PQ,MTSW4McQd7CbVtyjqoe9mw,4.0,2017,4,5
3,3,0q2W3-ieBUJWD5TTLKi3Ug,MTSW4McQd7CbVtyjqoe9mw,4.0,2016,4,5
4,4,z-yvbUGwFn8PAijEHdU_RA,MTSW4McQd7CbVtyjqoe9mw,5.0,2018,3,4


In [31]:
save_path = '/content/drive/MyDrive/Henry - Proyecto Final/Datos PF Henry/Unified/'
df_reviews.to_parquet(save_path+'df_reviews.parquet',index=False)